## Locating the data



-   google world ocean atlas e.g., [https://www.ncei.noaa.gov/products/world-ocean-atlas](https://www.ncei.noaa.gov/products/world-ocean-atlas)
-   locate dataset e.g., [https://www.ncei.noaa.gov/data/oceans/woa/WOA18/DATA/](https://www.ncei.noaa.gov/data/oceans/woa/WOA18/DATA/)
-   locate data for variable and spatial resolution [https://www.ncei.noaa.gov/data/oceans/woa/WOA18/DATA/phosphate/netcdf/all/1.00/](https://www.ncei.noaa.gov/data/oceans/woa/WOA18/DATA/phosphate/netcdf/all/1.00/)
-   Datasets are named in the following way:
    -   **00:** annual data
    -   **01 - 12:** monthly data
    -   **13 -16:** winter, spring, summer, autumn, 3 month each, starting in January
-   Depth Levels are as

| Depth|(m)|Level|Depth|(m)|Level|Depth|(m)|Level|Depth|(m)|Level|
|---|---|---|---|---|---|---|---|---|---|---|---|
| 0|1|250|11|1200|21|4500|31|||||
| 10|2|300|12|1300|22|5000|32|||||
| 20|3|400|13|1400|23|5500|33|||||
| 30|4|500|14|1500|24|6000|34|||||
| 50|5|600|15|1750|25|6500|35|||||
| 75|6|700|16|2000|26|7000|36|||||
| 100|7|800|17|2500|27|7500|37|||||
| 125|8|900|18|3000|28|8000|38|||||
| 150|9|1000|19|3500|29|8500|39|||||
| 200|10|1100|20|4000|30|9000|40|||||

-   variable names: typically with the `_an` postfix (e.g., for phosphate `p_an`" see [https://odv.awi.de/fileadmin/user_upload/odv/data/WOA09/woa09documentation.pdf>](https://odv.awi.de/fileadmin/user_upload/odv/data/WOA09/woa09documentation.pdf>)for more details
-   download the dataset into your local working directory
-   verify that it is there



## Importing libraries and dataset selection



-   [https://scitools-iris.readthedocs.io/en/v3.0.0/userguide/plotting_a_cube.html](https://scitools-iris.readthedocs.io/en/v3.0.0/userguide/plotting_a_cube.html)
-   [https://matplotlib.org/stable/tutorials/colors/colormaps.html](https://matplotlib.org/stable/tutorials/colors/colormaps.html)
-   



In [1]:
import os
# os.environ['PROJ_LIB'] = '/path/to/env/share/proj'
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import my_utils

species = "Oxygen"  #
depth_level = 0  # see documentation
time_frame = "annual"  # Match with your download
data_set = "WOA18"  # Match with your download
fig_size = (10, 7)  # in inches
color_scheme = "brewer_RdBu_11"

# load data
data, cb_label = my_utils.load_woa_data(species, depth_level)
# plot data
fig, ax = my_utils.make_map(data, ccrs.Robinson(), fig_size, cb_label, color_scheme)
# add title, save figure, and show plot
title = f"{data_set} {species} {depth_level} {time_frame}"
fig_name = f"{data_set}_{species}_{depth_level}_{time_frame}_{color_scheme}.png"
plt.title(title)
fig.savefig(fig_name)
plt.show()

This might result in a couple of warnings about units being wrong. Those can be ignored safely



## Some utility functions



### Getting the right data set



Here we download the data. Data locations change all the time, so you may have to adjust the urls from time to time.



In [1]:
def load_woa_data(species, depth_level):
    """setup the urls where the data can be found, do some sanity checks on the
    input, download data if necessary, otherwise use local copy, import data,
    and create data cube for a given variable name.

    Parameters:
      species = name of the data variable, e.g. Oxygen
      depth_level = depth relative to seas surface. See the WOA documentation 
    """
    import urllib.request
    import pathlib as pl
    import iris

    ou_url = "https://www.ncei.noaa.gov/data/oceans/woa/WOA18/DATA/o2sat/netcdf/all/1.00/woa18_all_O00_01.nc"

    p_url = "https://www.ncei.noaa.gov/data/oceans/woa/WOA18/DATA/phosphate/netcdf/all/1.00/woa18_all_p00_01.nc"

    o_url = "https://www.ncei.noaa.gov/data/oceans/woa/WOA18/DATA/oxygen/netcdf/all/1.00/woa18_all_o00_01.nc"

    T_url = "https://www.ncei.noaa.gov/data/oceans/woa/WOA18/DATA/temperature/netcdf/decav/1.00/woa18_decav_t00_01.nc"

    n_url = "https://www.ncei.noaa.gov/data/oceans/woa/WOA18/DATA/nitrate/netcdf/all/1.00/woa18_all_n00_01.nc"

    sp_dict = {
        "Phosphate": ["p_an", r"PO$_4$ [$\mu$mol/kg]", p_url],
        "Oxygen": ["o_an", r"O$_2$ [$\mu$mol/kg]", o_url],
        "Oxygen_utilisation": ["O_an", r"O$_2$ [$\mu$mol/kg]", o_url],
        "Nitrate": ["n_an", r"NO$_3$ [$\mu$mol/kg]", n_url],
        "Temperature": ["t_an", r"T [$^{\circ}$ C]", T_url],
    }

    if species in sp_dict:
        var_name = sp_dict[species][0]
        cb_label = sp_dict[species][1]
        url = sp_dict[species][2]
    else:
        raise ValueError(f"\n Unknown variable name {species}\n")

    fn: str = url.split("/")[-1]  # file name
    cwd: pl.Path = pl.Path.cwd()  # get the current working directory
    fqfn: pl.Path = pl.Path(f"{cwd}/{fn}")  # fully qualified file name

    if not fqfn.exists():  # download data if necessary
        urllib.request.urlretrieve(url, fqfn)
        if not fqfn.exists():  # verify download
            raise FileNotFoundError(f"Cannot find file {fqfn}")

    # import data
    cubes = iris.load(str(fqfn))  # iris cant handle the pathe object

    # find the cube id that matches the desired data
    for k, c in enumerate(cubes):
        if c.var_name == var_name:
            cube_id = k

    cube = cubes[cube_id]  # extract data
    c = cube[depth_level, 0, ...]  # Slice singleton time and first level.
    print("\n Data imported\n")
    return c, cb_label

# Out [1]:

### Making the plot



In [1]:
def make_map(cube, projection, figsize, cb_label, color_scheme):
    """Set up map parameters"""
    import iris.plot as iplt
    import matplotlib.cm as mpl_cm
    import matplotlib.pyplot as plt
    import cartopy.feature as cfeature

    # create figure instance
    fig, ax = plt.subplots(
        figsize=figsize, subplot_kw=dict(projection=projection), dpi=120
    )
    # plot data
    color_map = mpl_cm.get_cmap(color_scheme)
    cs = iplt.pcolormesh(cube, cmap=color_map)

    ax.coastlines()  # add coastlines
    ax.add_feature(cfeature.LAND, facecolor="0.75")  # mask land area

    # add color bar legend
    cbar = dict(
        extend="both", shrink=0.5, pad=0.02, orientation="horizontal", fraction=0.1
    )
    cb = fig.colorbar(cs, **cbar)
    cb.ax.set_xlabel(cb_label)

    return fig, ax